In [4]:
# 파일 읽어보기

## csv 파일 읽기
import csv

with open('월별_구별 교통사고.csv', 'r', encoding='utf-8') as file:
    csv_reader = csv.reader(file)
    data = list(csv_reader)

# data는 이제 CSV 파일의 내용을 담은 리스트입니다.
data[2:6]
print(data[3]) # 0:' ', 1:'year', 2:'month', 3-28:'사고수'
# 파악 완료

['', '2021', '12월', '66', '77', '127', '116', '188', '104', '105', '322', '78', '251', '181', '77', '109', '117', '121', '51', '94', '86', '77', '103', '108', '133', '51', '48', '79']


In [23]:
## table sql 만들기

# 기간(month_year) table
create_MY = """
create table MY(
  MY_id  varchar(20)  primary key,
  year varchar(20),
  month varchar(20)
)
"""

# gu table 
create_gu = """
create table gu(
  gu_id int auto_increment  primary key,
  gu_name varchar(20) unique
)
"""

#  사고발생 수(main)
create_number = """
create table number(
  number_id  int  auto_increment  primary key,
  gu_id  int,
  MY_id  varchar(20), 
  the_number_of_car_accident int,

  CONSTRAINT fk_number_gu FOREIGN KEY (gu_id) REFERENCES gu(gu_id) ON DELETE SET NULL,
  CONSTRAINT fk_number_MY FOREIGN KEY (MY_id) REFERENCES MY(MY_id) ON DELETE SET NULL
)
"""

In [24]:
import pymysql
import configparser as parser

# database의 내용을 parser로 읽어와서 코드에서는 정보가 드러나지 않도록 하고 싶습니다. 
# 그에 따른 조치사항으로 parser를 import 했습니다.  

try:
    conn = None
    
    props = parser.ConfigParser()  # parser 생성
    props.read("C:/Users/Playdata/Desktop/SK AI camp_6기_조하늘/SKN06-1st-7Team/월별_구별 교통사고 DB/config_car_accident_변형.ini") 
    # ini 파일 읽기 / 절대 경로로 변환
    config = props['skn06-1st-7team'] # section이름으로 읽기
    
    # 1. Database와 연결.
    conn = pymysql.connect(
        host=config['host'],    # DBMS 의 ip(host) : str
        port=3306,           # DBMS의 port 번호: int
        user=config['user'],        # username: str
        password=config['password'],    # password: str
        db=config['db']            #  연결할 Database이름: str
    )  # 연결 성공하면 연결된 DB와 관련 작업할 수있는 기능을 제공하는 Connection객체를 반환
    
    # 2. Connection을 사용해서 Cursor 객체 생성
    #    Cursor: sql 처리를 하는 기능을 제공.( sql 전송하고 처리결과를 받을 때까지를 관리)
    cursor = conn.cursor()

    cursor.execute("drop table if exists number")
    cursor.execute("drop table if exists gu")
    cursor.execute("drop table if exists MY")
    cursor.execute(create_MY)
    cursor.execute(create_gu)
    cursor.execute(create_number)
    

finally:
    # 4. 연결닫기(끊기)
    if conn: # conn != None
        cursor.close() # 4-1. cursor 연결 닫기
        conn.close()  # 4-2. connection 연결 닫기

print('실행 완료')

실행 완료


In [25]:
# 파일 형식 확인
print(data[2])
print("-"*59)
print(data[3])
print("-"*59)
print(len(data))
print("data[0]은 공백, data[1]은 서울표시, data[2]는 구표시, data[3]은 월 및 사고수 표시")

['', '시군구', '시군구', '종로구', '마포구', '강서구', '구로구', '영등포구', '동작구', '관악구', '강남구', '강동구', '송파구', '서초구', '중구', '양천구', '중랑구', '노원구', '광진구', '강북구', '금천구', '용산구', '성동구', '동대문구', '성북구', '도봉구', '은평구', '서대문구']
-----------------------------------------------------------
['', '2021', '12월', '66', '77', '127', '116', '188', '104', '105', '322', '78', '251', '181', '77', '109', '117', '121', '51', '94', '86', '77', '103', '108', '133', '51', '48', '79']
-----------------------------------------------------------
28
data[0]은 공백, data[1]은 서울표시, data[2]는 구표시, data[3]은 월 및 사고수 표시


In [26]:
# MY table 받기
# 서울 데이터만 불러오고 싶다. 각 구 이름이 있는 위치는 data[i][1] i > 1 이다. 그렇다면 data[i][1] == "서울" 인 것만 담는다.

# create_MY = """
# create table MY(
#   MY_id  varchar(20)  primary key,
#   year varchar(20),
#   month varchar(20)
# )
# """

data_MY = [] # sql에 최종적으로 넣을 값.

for i in range(len(data)-3): # 3행부터 년월이 표기되기 때문
    # data 분류 필요
    insert_list = [] # 변환되는 값.
    year:str = None
    month:str = None
    year = data[i+3][1] 
    month = data[i+3][2]
    month = month[:-1] # 월 지우기 
    MY_id = year + "-" + month
    insert_list.append(MY_id)
    insert_list.append(year)
    insert_list.append(month)
    data_MY.append(list(insert_list))


 # insert 하기
insert_sql = """insert into MY(MY_id, year, month) values(%s, %s, %s)"""

with pymysql.connect(host=config['host'],
                     port=3306, user=config['user'],
                     password=config['password'],
                     db=config['db']) as conn:
    with conn.cursor() as cursor:
        cnt = cursor.executemany(insert_sql, data_MY)  # executemany() 결과 행수 반환.
        conn.commit()
        

print("insert된 총 행수:", cnt)

insert된 총 행수: 25


In [27]:
print(data[2])
print(len(data[2]))

['', '시군구', '시군구', '종로구', '마포구', '강서구', '구로구', '영등포구', '동작구', '관악구', '강남구', '강동구', '송파구', '서초구', '중구', '양천구', '중랑구', '노원구', '광진구', '강북구', '금천구', '용산구', '성동구', '동대문구', '성북구', '도봉구', '은평구', '서대문구']
28


In [28]:
# gu table 받기
# 서울 데이터만 불러오고 싶다. 각 구 이름이 있는 위치는 data[i][1] i > 1 이다. 그렇다면 data[i][1] == "서울" 인 것만 담는다.

# 참고
# gu table 
# create_gu = """
# create table gu(
#   gu_id int auto_increment  primary key,
#   gu_name varchar(20) unique
# )
# """

data_gu = [] # sql에 최종적으로 넣을 값.


for i in range(len(data[2])-3): # 28개 
    data_gu.append(data[2][i+3])

print(data_gu) # 제대로 입력되었는지 확인
print(len(data_gu)) # 길이 확인

# insert 하기
insert_sql = """insert into gu(gu_name) values(%s)"""

with pymysql.connect(host=config['host'],
                     port=3306, user=config['user'],
                     password=config['password'],
                     db=config['db']) as conn:
    with conn.cursor() as cursor:
        cnt = cursor.executemany(insert_sql, data_gu)  # executemany() 결과 행수 반환.
        conn.commit()
        

print("insert된 총 행수:", cnt)

['종로구', '마포구', '강서구', '구로구', '영등포구', '동작구', '관악구', '강남구', '강동구', '송파구', '서초구', '중구', '양천구', '중랑구', '노원구', '광진구', '강북구', '금천구', '용산구', '성동구', '동대문구', '성북구', '도봉구', '은평구', '서대문구']
25
insert된 총 행수: 25


In [29]:
print(data[3])

['', '2021', '12월', '66', '77', '127', '116', '188', '104', '105', '322', '78', '251', '181', '77', '109', '117', '121', '51', '94', '86', '77', '103', '108', '133', '51', '48', '79']


In [33]:
# number table 받기

# 참고
# #  사고발생 수(main)
# create_number = """
# create table number(
#   number_id  int  auto_increment  primary key,
#   gu_id  int,
#   MY_id  varchar(20), 
#   the_number_of_car_accident int,

#   CONSTRAINT fk_number_gu FOREIGN KEY (gu_id) REFERENCES gu(gu_id) ON DELETE SET NULL,
#   CONSTRAINT fk_number_MY FOREIGN KEY (MY_id) REFERENCES MY(MY_id) ON DELETE SET NULL
# )
# """
data_number = [] # sql에 최종적으로 넣을 값.

for i in range(len(data)-3): # 기간 할당
    insurt_list = []
    year:str = None
    month:str = None
    year = data[i+3][1] 
    month = data[i+3][2]
    month = month[:-1] # 월 지우기 
    MY_id = year + "-" + month
    insert_list.append(MY_id) # MY_id 할당
    
    for j in range(len(data)-3): # 25번 입력, 각 구의 사고수 할당
        if len(insurt_list) > 2: # 할당된 것 삭제
            insurt_list.pop()
            insurt_list.pop()
            insurt_list.append(j+1) # gu_id 할당
            insurt_list.append(data[i+3][j+3]) # the_number_of_car_accident 할당
            data_number.append(list(insurt_list))
        else: # 첫번째 시행
            insurt_list.append(j+1) # gu_id 할당
            insurt_list.append(data[i+3][j+3])
            data_number.append(list(insurt_list))


data_number[:2]
# # insert 하기
# insert_sql = """insert into number(MY_id, gu_id, the_number_of_car_accident) values(%s, %s, %s)"""

# with pymysql.connect(host=config['host'],
#                      port=3306, user=config['user'],
#                      password=config['password'],
#                      db=config['db']) as conn:
#     with conn.cursor() as cursor:
#         cnt = cursor.executemany(insert_sql, data_number)  # executemany() 결과 행수 반환.
#         conn.commit()
        
# # print("insert된 총 행수:", cnt)
# print("실행 완료")
# print(len(data_number)) # 25*25 = 월 수 * 구 수 / 맞다. 

[[1, '66'], [1, '66', 2, '77']]